<a href="https://colab.research.google.com/github/nortonvanz/PySpark-Basics/blob/main/notebooks/Case_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()


In [5]:
#imports
import random

# Load Datasets

In [6]:
#Carrega datasets do github:
!wget --quiet --show-progress https://raw.githubusercontent.com/iara/desafio_cogna/main/dataset/tabela_profile.csv
!wget --quiet --show-progress https://raw.githubusercontent.com/iara/desafio_cogna/main/dataset/tabela_carro.csv
!wget --quiet --show-progress https://raw.githubusercontent.com/iara/desafio_cogna/main/dataset/tabela_emprego.csv

tabela_profile.csv  100%[===================>]   1.30M  --.-KB/s    in 0.08s   
tabela_carro.csv    100%[===================>] 159.33K  --.-KB/s    in 0.04s   
tabela_emprego.csv  100%[===================>] 196.54K  --.-KB/s    in 0.04s   


In [7]:
#arquivos salvos no Colab
!ls

sample_data		   spark-3.1.2-bin-hadoop2.7.tgz  tabela_emprego.csv
spark-3.1.2-bin-hadoop2.7  tabela_carro.csv		  tabela_profile.csv


In [8]:
df_pessoa = spark.read.csv("tabela_profile.csv", sep=",", inferSchema=True, header=True)
df_carro = spark.read.csv("tabela_carro.csv", sep=",", inferSchema=True, header=True)
df_emprego = spark.read.csv("tabela_emprego.csv", inferSchema=True, header=True)

In [9]:
df_emprego.printSchema()

root
 |-- id_profile: integer (nullable = true)
 |-- cargo: string (nullable = true)
 |-- empresa: string (nullable = true)
 |-- id_emprego: integer (nullable = true)



In [39]:
df_pessoa.show(3)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+
|    579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04|
|    739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17|
|     75467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+
only showing top 3 rows



In [11]:
df_carro.show(3)

+----------+-----------+--------------+--------+
|id_profile|placa_carro|data_aquisicao|id_carro|
+----------+-----------+--------------+--------+
|    591942|   ZHT-2654|    2001-03-06|   73634|
|    381762|   LEL-7635|    1999-01-29|   39149|
|    653260|   KUL-6169|    1999-06-13|   48149|
+----------+-----------+--------------+--------+
only showing top 3 rows



In [12]:
df_emprego.show(3)

+----------+------------------+------------------+----------+
|id_profile|             cargo|           empresa|id_emprego|
+----------+------------------+------------------+----------+
|     33062|Engenheiro químico|      da Conceição|     38503|
|     97704|          Contador|Sales Caldeira S/A|     79501|
|     19073|          Marechal|            Fogaça|     33353|
+----------+------------------+------------------+----------+
only showing top 3 rows



# Questões de Negócio

## 1 Quantas porcento das pessoas  empregadas?

In [ ]:
#juntar df_pessoa com df_emprego pelo id_profile
df_pessoa = df_pessoa.join(df_emprego, ["id_profile"], "left")

In [ ]:
df_pessoa.show(5)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+
|    579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|
|    739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|
|     75467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|
|    233935|  helenaalves|Dr. João Pedro Fe...|   H|Loteamento Barbos...|mariamelo@hotmail...| 1939-02-04| null|   null|      null|
|     42638|  alexandre30|     Juliana Peixoto|   M|Rua de Castro  41...|   

In [ ]:
df_pessoa.createOrReplaceTempView("PESSOA")

In [ ]:
#SPARK SQL
spark.sql('''
          SELECT
            CASE WHEN CARGO IS NULL THEN 'NÃO' ELSE 'SIM' END AS EMPREGADO,
            COUNT(ID_PROFILE) AS PESSOAS
          FROM PESSOA
          GROUP BY EMPREGADO

        ''').show(5)

+---------+-------+
|EMPREGADO|PESSOAS|
+---------+-------+
|      NÃO|   6485|
|      SIM|   3521|
+---------+-------+



In [ ]:
# % de empregados:
3521 / 10006 * 100

35.188886667999206

In [ ]:
# SQL FUNCTION

#criar campo tem_emprego, com 0 ou 1:
df_pessoa2 = df_pessoa.withColumn("tem_emprego", when(col("id_emprego").isNotNull(), 1).when(col("id_emprego").isNull(), 0))

In [ ]:
df_pessoa2.show(3)

+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----------+
|id_profile|     username|                name|sexo|            endereço|               email|aniversario|cargo|empresa|id_emprego|tem_emprego|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----------+
|    579363|       ynunes|    Augusto Ferreira|   H|Trecho Leandro Sa...|ana-julia59@ig.co...| 1934-05-04| null|   null|      null|          0|
|    739543|kamillyvieira|        Helena Viana|   M|Trevo Nogueira  4...|vianamaria-julia@...| 2000-07-17| null|   null|      null|          0|
|     75467| araujofelipe|Srta. Amanda Teix...|   M|Condomínio de Nov...|   qcastro@gmail.com| 1936-07-25| null|   null|      null|          0|
+----------+-------------+--------------------+----+--------------------+--------------------+-----------+-----+-------+----------+-----

In [ ]:
tem_emprego = df_pessoa2.where(col("tem_emprego") == 1).count()
total_pessoas = df_pessoa2.count()

porcentagem=tem_emprego/total_pessoas*100

print(f"tem_emprego={tem_emprego}")
print(f"total_pessoas={total_pessoas}")
print(f"quem tem emprego={porcentagem}%")

tem_emprego=3521
total_pessoas=10006
quem tem emprego=35.188886667999206%


## 2 Criar os salarios anuais dos empregados de forma randomica, sendo que o minimo é 1000

In [ ]:
@udf
def gera_salarios():
  return random.randint(1000, 1000000)

df_emprego = df_emprego.withColumn("salario", gera_salarios())

In [ ]:
df_emprego.show(5)

+----------+--------------------+------------------+----------+-------+
|id_profile|               cargo|           empresa|id_emprego|salario|
+----------+--------------------+------------------+----------+-------+
|     33062|  Engenheiro químico|      da Conceição|     38503| 374190|
|     97704|            Contador|Sales Caldeira S/A|     79501| 598622|
|     19073|            Marechal|            Fogaça|     33353| 303848|
|     18918|               Cumim|            da Luz|     24143| 770022|
|     61048|Conferente de exp...|        Cavalcanti|     86763| 971290|
+----------+--------------------+------------------+----------+-------+
only showing top 5 rows



In [ ]:
df_emprego.show(5)

+----------+--------------------+------------------+----------+-------+
|id_profile|               cargo|           empresa|id_emprego|salario|
+----------+--------------------+------------------+----------+-------+
|     33062|  Engenheiro químico|      da Conceição|     38503| 792662|
|     97704|            Contador|Sales Caldeira S/A|     79501| 474064|
|     19073|            Marechal|            Fogaça|     33353| 726497|
|     18918|               Cumim|            da Luz|     24143| 327707|
|     61048|Conferente de exp...|        Cavalcanti|     86763| 881921|
+----------+--------------------+------------------+----------+-------+
only showing top 5 rows



## 3 Qual a média salarial de cada cargo?

In [ ]:
df_emprego.show(10)

+----------+--------------------+--------------------+----------+-------+
|id_profile|               cargo|             empresa|id_emprego|salario|
+----------+--------------------+--------------------+----------+-------+
|     33062|  Engenheiro químico|        da Conceição|     38503| 183523|
|     97704|            Contador|  Sales Caldeira S/A|     79501| 790040|
|     19073|            Marechal|              Fogaça|     33353| 945347|
|     18918|               Cumim|              da Luz|     24143| 112267|
|     61048|Conferente de exp...|          Cavalcanti|     86763| 685395|
|     75089|Técnico em docume...| Moraes Moreira S.A.|     35470| 674510|
|     11338| Polidor de produção|               Pinto|     48914| 373223|
|     53918|        Carnavalesco|        da Rosa - ME|     71077| 941089|
|     11512|          Cartunista|Nunes Moreira e F...|     28114| 518765|
|     29897|Agente penitenciário|              Aragão|     12559| 318694|
+----------+--------------------+-----

In [ ]:
#SQL FUNCTIONS

#para um cargo:
df_emprego.filter(col("cargo") == "Engenheiro químico").show()

+----------+------------------+------------------+----------+-------+
|id_profile|             cargo|           empresa|id_emprego|salario|
+----------+------------------+------------------+----------+-------+
|     33062|Engenheiro químico|      da Conceição|     38503| 803262|
|     57836|Engenheiro químico|Barbosa Gomes - ME|     90544| 814735|
|     55758|Engenheiro químico|              Dias|     90896| 393261|
|     59095|Engenheiro químico|             Alves|     47752| 772929|
|    806335|Engenheiro químico|          Silveira|     30103| 683411|
|    698440|Engenheiro químico|             Lopes|     10085| 796995|
|    542447|Engenheiro químico|       Costela S/A|      8995| 832955|
|    403934|Engenheiro químico|           Rezende|     83488|   9623|
|    230830|Engenheiro químico|            Vieira|     64458|  11798|
|    827496|Engenheiro químico|             Gomes|     44632| 281610|
|    216314|Engenheiro químico|           Barbosa|     14463| 259214|
|    205086|Engenhei

In [ ]:
# Média salarial por cargo:
df_emprego.groupBy("cargo").agg(avg("salario").alias("media_salarial")).show(10)

+--------------------+------------------+
|               cargo|    media_salarial|
+--------------------+------------------+
|Engenheiro de erg...|          472838.6|
|Auxiliar de repro...|          187392.0|
|        Neurologista|448546.85714285716|
|             Tenente|         561940.75|
|          Gastrônomo| 236349.2857142857|
|           Bailarina|464279.28571428574|
|Profissional de r...|479660.45454545453|
|         Pastilheiro| 623485.7272727273|
|             Lavador|          323248.4|
|Classificador con...|          497132.2|
+--------------------+------------------+
only showing top 10 rows



In [ ]:
#SPARK SQL
df_emprego.createOrReplaceTempView("EMPREGO")

In [ ]:
#SPARK SQL
spark.sql('''
          SELECT
            CARGO,
            AVG(SALARIO) AS MEDIA_SALARIAL
          FROM EMPREGO
          WHERE CARGO = 'Neurologista'
          GROUP BY CARGO

        ''').show(100)

+------------+-----------------+
|       CARGO|   MEDIA_SALARIAL|
+------------+-----------------+
|Neurologista|507268.5714285714|
+------------+-----------------+



In [ ]:
from pyspark.sql.window import Window

windows  = Window.partitionBy("cargo")

df_emprego = df_emprego.withColumn("media_salarial", avg(col("salario")).over(windows))

## 4 Gerar um numero sequencial por por cargos e eliminar todos as linhas maiores que 1.

In [ ]:
windows  = Window.partitionBy("cargo").orderBy(col("salario").asc())
df_emprego = df_emprego.withColumn("row_number", row_number().over(windows))

df_emprego = df_emprego.filter(col("row_number") == 1)

NameError: ignored

In [ ]:
df_emprego.show(15)

+----------+--------------------+--------------------+----------+
|id_profile|               cargo|             empresa|id_emprego|
+----------+--------------------+--------------------+----------+
|     33062|  Engenheiro químico|        da Conceição|     38503|
|     97704|            Contador|  Sales Caldeira S/A|     79501|
|     19073|            Marechal|              Fogaça|     33353|
|     18918|               Cumim|              da Luz|     24143|
|     61048|Conferente de exp...|          Cavalcanti|     86763|
|     75089|Técnico em docume...| Moraes Moreira S.A.|     35470|
|     11338| Polidor de produção|               Pinto|     48914|
|     53918|        Carnavalesco|        da Rosa - ME|     71077|
|     11512|          Cartunista|Nunes Moreira e F...|     28114|
|     29897|Agente penitenciário|              Aragão|     12559|
|     13709|Intérprete de Bíb...|     Campos e Filhos|     72721|
|     31423|Engenheiro de hor...|               Costa|     80339|
|      629

## 5 Quem é a pessoa mais jovem a ter um carro?

In [ ]:
df_carro2 = df_pessoa.join(df_carro, ["id_profile"], "inner")

In [ ]:
df_carro2.withColumn("aniversario", to_date(col("aniversario"),"yyyy-MM-dd"))

df_carro2.orderBy(col("aniversario").desc()).show(1)

NameError: ignored

In [ ]:
#df_carro2.select(max(col("aniversario"))).show(1)

df_carro2.groupBy("sexo").agg(max(col("aniversario"))).show()




NameError: ignored

## 6 Normalizar o id_profile para 8 digitos nos dataframes

In [ ]:
df_emprego = df_emprego.withColumn("id_profile", lpad(col("id_profile"),8, '0'))
df_carro = df_carro.withColumn("id_profile", lpad(col("id_profile"),8, '0'))
df_pessoa = df_pessoa.withColumn("id_profile", lpad(col("id_profile"),8, '0'))

## 7 Remover o acento dos nomes no dataframe de pessoa.

In [ ]:
from unidecode import unidecode
print(unidecode('Arrepieí'))

Arrepiei


In [ ]:

df_pessoa.rdd.map(lambda x: x.encode("ascii","ignore"))

PythonRDD[74] at RDD at PythonRDD.scala:53

In [ ]:
@udf
def unicode_name(name):
  return unidecode(name)

df_pessoa.withColumn("name", unicode_name(col("name"))).show()



# unicodeUDF = udf(lambda z: unidecode(z))
# df_pessoa.select(unicodeUDF(col("name"))).show()

NameError: ignored

## 8 Quantas pessoas nasceram em cada década?

In [ ]:
df2 = df_pessoa.groupBy(year(col("aniversario")).alias("ano_nascimento")).agg(count("id_profile").alias("count_nascimento"))

NameError: ignored

In [ ]:
df2 = df2.orderBy(col("ano_nascimento").asc())

NameError: ignored

In [ ]:
ano_inicial = 1900
dict_decada = {}
for i in range(0, 12):
  ano_final = ano_inicial + 9
  decada_list = [ano_inicial, ano_final]
  key_decada = f"{ano_inicial}-{ano_final}"
  ano_inicial = ano_final + 1
  dict_decada[key_decada] = decada_list

In [ ]:
dict_decada

{'1900-1909': [1900, 1909],
 '1910-1919': [1910, 1919],
 '1920-1929': [1920, 1929],
 '1930-1939': [1930, 1939],
 '1940-1949': [1940, 1949],
 '1950-1959': [1950, 1959],
 '1960-1969': [1960, 1969],
 '1970-1979': [1970, 1979],
 '1980-1989': [1980, 1989],
 '1990-1999': [1990, 1999],
 '2000-2009': [2000, 2009],
 '2010-2019': [2010, 2019]}

In [ ]:
@udf
def find_decade(ano_target):
  for key in dict_decada.keys():
    anos = dict_decada[key]
    ano_inicial = anos[0]
    ano_final = anos[1]
    if(ano_target >= ano_inicial and ano_target <= ano_final):
      return key
  return 0


NameError: ignored

In [ ]:
#1905%100

#ps.date_range(start='1/1/2018', periods=5, freq='M')
import pyspark.pandas as ps

ModuleNotFoundError: ignored

In [ ]:
df3=df2.withColumn("decada", find_decade(col("ano_nascimento")))

In [ ]:
df3.show()

In [ ]:
df2=df2.withColumn("ano_nascimento", when(col("ano_nascimento").isNull(), 0).otherwise(col("ano_nascimento")))

In [ ]:
df2.show()

In [ ]:
df4=df2.withColumn("decada", when((col("ano_nascimento") >=1900) & (col("ano_nascimento") <1910), 00)
                         .when((col("ano_nascimento") >=1910) & (col("ano_nascimento") <1920), 10)
                         .when((col("ano_nascimento") >=1920) & (col("ano_nascimento") <1930), 20)
                         .when((col("ano_nascimento") >=1930) & (col("ano_nascimento") <1940), 30)
                         .when((col("ano_nascimento") >=1940) & (col("ano_nascimento") <1950), 40)
                         .when((col("ano_nascimento") >=1950) & (col("ano_nascimento") <1960), 50)
                         .when((col("ano_nascimento") >=1960) & (col("ano_nascimento") <1970), 60)
                         .when((col("ano_nascimento") >=1970) & (col("ano_nascimento") <1980), 70)
                         .when((col("ano_nascimento") >=1980) & (col("ano_nascimento") <1990), 80)
                         .when((col("ano_nascimento") >=1990) & (col("ano_nascimento") <2000), 90)
                         .when((col("ano_nascimento") >=2000) & (col("ano_nascimento") <2010), 200)
                         .when((col("ano_nascimento") >=2010) & (col("ano_nascimento") <2020), 210)
                         .otherwise(3333)
              )

In [ ]:
df5=df4.groupBy(col("decada")).agg(sum(col("count_nascimento").alias("count_nascimento_decada")))

In [ ]:
df4.filter(col("ano_nascimento")> 0).show()

## 9 Qual a década que que há mais pessoas empregadas?


In [ ]:
df_pessoa2 = df_pessoa.filter((col("tem_emprego") == 1) & (col("aniversario").isNotNull()))

In [ ]:
df_pessoa2 = df_pessoa2.groupBy(year(col("aniversario")).alias("ano_nascimento")).agg(count("id_profile").alias("count_nascimento"))
df_pessoa2 = df_pessoa2.filter(col("ano_nascimento").isNotNull())
df_pessoa2 = df_pessoa2.orderBy(col("ano_nascimento").asc())


df_pessoa2 = df_pessoa2.withColumn("decada", when((col("ano_nascimento") >=1900) & (col("ano_nascimento") <1910), 00)
                         .when((col("ano_nascimento") >=1910) & (col("ano_nascimento") <1920), 10)
                         .when((col("ano_nascimento") >=1920) & (col("ano_nascimento") <1930), 20)
                         .when((col("ano_nascimento") >=1930) & (col("ano_nascimento") <1940), 30)
                         .when((col("ano_nascimento") >=1940) & (col("ano_nascimento") <1950), 40)
                         .when((col("ano_nascimento") >=1950) & (col("ano_nascimento") <1960), 50)
                         .when((col("ano_nascimento") >=1960) & (col("ano_nascimento") <1970), 60)
                         .when((col("ano_nascimento") >=1970) & (col("ano_nascimento") <1980), 70)
                         .when((col("ano_nascimento") >=1980) & (col("ano_nascimento") <1990), 80)
                         .when((col("ano_nascimento") >=1990) & (col("ano_nascimento") <2000), 90)
                         .when((col("ano_nascimento") >=2000) & (col("ano_nascimento") <2010), 200)
                         .when((col("ano_nascimento") >=2010) & (col("ano_nascimento") <2020), 210)
                         .otherwise(3333)
              )

In [ ]:
df_pessoa2=df_pessoa2.groupBy(col("decada")).agg(sum(col("count_nascimento")).alias("count_nascimento_decada"))

In [ ]:
df_pessoa2.orderBy(col("count_nascimento_decada").desc()).show(1)